In [4]:
# Imports
import os, json
import numpy as np
import pandas as pd
import plotly.express as px

In [5]:
#### Loading CSV file

# ----------------------------
# Config
# ----------------------------
CSV_PATH = "results/koltsov3_4different_perm1_results.csv"  # input data
OUT_HTML = "results/koltsov3_4different_perm1_report.html"  # single report file
HEATMAP_NS = [20, 24, 25, 28, 30]                  # n values to snapshot (adjust)
MAX_FACET_ROWS = 10                            # avoid gigantic facets; trims d panels

N_MIN = 12        # start n shown
N_MAX = 30        # end n shown
DEFAULT_KS = 8    # how many ks to show by default in the diameter plot


os.makedirs(os.path.dirname(OUT_HTML), exist_ok=True)

# ----------------------------
# Load + parse
# ----------------------------
df = pd.read_csv(CSV_PATH)

def parse_dict(s):
    # JSON dict with string keys -> dict with int keys
    dct = json.loads(s)
    return {int(k): dct[k] for k in dct}

df["diam_dict"] = df["diameters"].apply(parse_dict)
df["last_layer_dict"] = df["last_layers"].apply(parse_dict)
df["states_dict"] = df["total_states"].apply(parse_dict)
df["n_list"] = df["n_values"].apply(json.loads)

In [6]:
# Long-form tables for plotting
rows = []
rows_ll = []
rows_states = []
for _, r in df.iterrows():
    k = int(r["k_param"])
    d = int(r["d_param"])
    perm_type = int(r["perm_type"])
    coset = str(r["coset"])
    for n, diam in r["diam_dict"].items():
        rows.append({"k": k, "d": d, "perm_type": perm_type, "coset": coset, "n": int(n), "diameter": float(diam)})
        # last layer (if present)
        if int(n) in r["last_layer_dict"]:
            rows_ll.append({"k": k, "d": d, "perm_type": perm_type, "coset": coset,
                            "n": int(n), "diameter": float(diam), "last_layer_size": float(r["last_layer_dict"][int(n)])})
        # states (if present)
        if int(n) in r["states_dict"]:
            rows_states.append({"k": k, "d": d, "perm_type": perm_type, "coset": coset,
                                "n": int(n), "total_states": float(r["states_dict"][int(n)])})

long_df = pd.DataFrame(rows)
ll_df = pd.DataFrame(rows_ll)
states_df = pd.DataFrame(rows_states)

# Helpful derived columns
long_df["n_mod_2"] = (long_df["n"] % 2).astype(int)
long_df["diam_over_n2"] = long_df["diameter"] / (long_df["n"] ** 2)

# Optionally trim number of d panels for readability
all_ds = sorted(long_df["d"].unique())
if len(all_ds) > MAX_FACET_ROWS:
    ds_keep = all_ds[:MAX_FACET_ROWS]
    long_df_trim = long_df[long_df["d"].isin(ds_keep)].copy()
else:
    long_df_trim = long_df

# ----------------------------
# Helpers: build heatmap grid at fixed n
# ----------------------------
def heatmap_at_n(target_n: int):
    sub = long_df[long_df["n"] == target_n].copy()
    if sub.empty:
        return None

    # pivot to d x k
    pivot = sub.pivot_table(index="d", columns="k", values="diameter", aggfunc="mean")
    # ensure sorted axes
    pivot = pivot.sort_index().sort_index(axis=1)

    fig = px.imshow(
        pivot.values,
        x=pivot.columns.astype(int),
        y=pivot.index.astype(int),
        labels={"x": "k_param", "y": "d_param", "color": "Diameter"},
        title=f"Diameter heatmap at n={target_n}",
        aspect="auto"
    )
    fig.update_layout(margin=dict(l=40, r=20, t=60, b=40))
    return fig


In [7]:
# ----------------------------
# Create figures
# ----------------------------
figs = []

# Filter to a window of n to reduce clutter
long_df_window = long_df_trim[(long_df_trim["n"] >= N_MIN) & (long_df_trim["n"] <= N_MAX)].copy()
ll_df_window = ll_df[(ll_df["n"] >= N_MIN) & (ll_df["n"] <= N_MAX)].copy() if not ll_df.empty else ll_df
states_df_window = states_df[(states_df["n"] >= N_MIN) & (states_df["n"] <= N_MAX)].copy() if not states_df.empty else states_df

# 1) Heatmaps for chosen n values (unchanged)
for n0 in HEATMAP_NS:
    fig = heatmap_at_n(n0)
    if fig is not None:
        figs.append(("Heatmap", fig))

# ----------------------------
# 2) Diameter vs n (faceted by d, colored by k) + k selector
# ----------------------------

# Choose default visible ks (small subset to avoid spaghetti)
all_ks = sorted(long_df_window["k"].unique())
default_ks = all_ks[:min(DEFAULT_KS, len(all_ks))]

fig_diam_vs_n = px.line(
    long_df_window.sort_values(["d", "k", "n"]),
    x="n",
    y="diameter",
    color="k",
    facet_row="d",
    line_group="k",
    title=f"Diameter vs n (n in [{N_MIN},{N_MAX}]) — legend click hides/shows k"
)

# Limit x-range explicitly (even though we filtered, this helps)
fig_diam_vs_n.update_xaxes(range=[N_MIN, N_MAX])

# --- Make only DEFAULT_KS visible initially (others hidden but selectable via legend/dropdown) ---
for tr in fig_diam_vs_n.data:
    # trace.name is the k value as a string
    try:
        k_val = int(tr.name)
    except Exception:
        continue
    if k_val not in default_ks:
        tr.visible = "legendonly"

# --- Add dropdown presets for k selection ---
def vis_mask_for_ks(ks_to_show):
    # returns a list aligned with fig_diam_vs_n.data: True/legendonly for each trace
    mask = []
    for tr in fig_diam_vs_n.data:
        try:
            k_val = int(tr.name)
            mask.append(True if k_val in ks_to_show else "legendonly")
        except Exception:
            mask.append(True)
    return mask

even_ks = [k for k in all_ks if k % 2 == 0]
odd_ks = [k for k in all_ks if k % 2 == 1]

buttons = [
    dict(label=f"Default (first {len(default_ks)})", method="update",
         args=[{"visible": vis_mask_for_ks(default_ks)}]),
    dict(label="Show ALL k", method="update",
         args=[{"visible": [True] * len(fig_diam_vs_n.data)}]),
    dict(label="Hide ALL k", method="update",
         args=[{"visible": ["legendonly"] * len(fig_diam_vs_n.data)}]),
    dict(label="Even k only", method="update",
         args=[{"visible": vis_mask_for_ks(even_ks)}]),
    dict(label="Odd k only", method="update",
         args=[{"visible": vis_mask_for_ks(odd_ks)}]),
]

fig_diam_vs_n.update_layout(
    height=250 * len(sorted(long_df_window["d"].unique())),
    margin=dict(l=40, r=20, t=80, b=40),
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        x=1.02, xanchor="left",
        y=1.0, yanchor="top",
        buttons=buttons
    )],
    legend_title_text="k (click to toggle)"
)

figs.append(("Diameter vs n", fig_diam_vs_n))

# 3) Parity (n mod 2) scatter (use windowed data)
fig_parity = px.scatter(
    long_df_window,
    x="n",
    y="diameter",
    color="n_mod_2",
    facet_row="d",
    symbol="n_mod_2",
    title=f"Parity test: diameter vs n (n in [{N_MIN},{N_MAX}])"
)
fig_parity.update_layout(height=250 * len(sorted(long_df_window["d"].unique())),
                         margin=dict(l=40, r=20, t=60, b=40))
figs.append(("Parity", fig_parity))

# 4) Last-layer size vs diameter (use windowed data)
if not ll_df_window.empty:
    fig_ll = px.scatter(
        ll_df_window,
        x="diameter",
        y="last_layer_size",
        color="d",
        hover_data=["k", "n"],
        title=f"Last-layer size vs diameter (n in [{N_MIN},{N_MAX}])"
    )
    fig_ll.update_layout(margin=dict(l=40, r=20, t=60, b=40))
    figs.append(("Last layer", fig_ll))

# 5) Normalized diameter vs d at max n (unchanged logic, but keep as-is)
n_max = int(long_df["n"].max())
sub_norm = long_df[long_df["n"] == n_max].copy()
if not sub_norm.empty:
    fig_norm = px.scatter(
        sub_norm,
        x="d",
        y="diam_over_n2",
        color="k",
        title=f"Normalized diameter at n={n_max}: diameter / n² vs d (color=k)"
    )
    fig_norm.update_layout(margin=dict(l=40, r=20, t=60, b=40))
    figs.append(("Normalized", fig_norm))

# Total states vs n (use windowed data)
if not states_df_window.empty:
    fig_states = px.line(
        states_df_window.sort_values(["d", "k", "n"]),
        x="n",
        y="total_states",
        color="k",
        facet_row="d",
        line_group="k",
        title=f"Total states vs n (n in [{N_MIN},{N_MAX}])",
        log_y=True
    )
    fig_states.update_layout(height=250 * len(sorted(states_df_window["d"].unique()[:MAX_FACET_ROWS])),
                             margin=dict(l=40, r=20, t=60, b=40))
    figs.append(("Total states", fig_states))


# ----------------------------
# Write single HTML report
# ----------------------------
# Use Plotly JS once (CDN) for the first figure, then omit
divs = []
for i, (name, fig) in enumerate(figs):
    div = fig.to_html(full_html=False, include_plotlyjs=("cdn" if i == 0 else False))
    divs.append(f"<h2 style='font-family: sans-serif;'>{name}</h2>\n{div}")

summary = f"""
<ul style="font-family: sans-serif; line-height: 1.4;">
  <li><b>Source CSV:</b> {CSV_PATH}</li>
  <li><b>Rows (parameter sets):</b> {len(df)}</li>
  <li><b>Points (n,diam) total:</b> {len(long_df)}</li>
  <li><b>Unique k:</b> {len(sorted(df['k_param'].unique()))} |
      <b>Unique d:</b> {len(sorted(df['d_param'].unique()))} |
      <b>Coset(s):</b> {", ".join(sorted(df['coset'].astype(str).unique()))}</li>
  <li><b>Heatmap snapshots at n:</b> {HEATMAP_NS}</li>
</ul>
"""

html = f"""
<!doctype html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Koltsov3 Diameter Report</title>
</head>
<body style="margin: 24px;">
  <h1 style="font-family: sans-serif;">Koltsov3 Diameter / Growth Report</h1>
  {summary}
  {'<hr/>'.join(divs)}
</body>
</html>
"""

with open(OUT_HTML, "w", encoding="utf-8") as f:
    f.write(html)

print("Wrote report:", OUT_HTML)

Wrote report: results/koltsov3_4different_perm1_report.html
